In [ ]:
import json
import pandas as pd

In [ ]:
from webpage.appcreator.import_utils import fetch_models
from filechecker.filechecker_utils import filechecker_to_df, find_matching_objects

In [ ]:
# file = r"/mnt/OeAW_Projekte03/OREA-EGYPT_Puzzle4D/ARCHE-preparation/checkReports/2019_11_26_Puzzle4d/fileList.json"
# file = r"/home/csae8092/repos/p4d/data/fileList.json"
file = r"/home/csae8092/repos/p4d/fileList.json"  # add missing dbs 2020-12-03

In [ ]:
# data = json.load(open(file, "r", encoding="utf-8"))

In [ ]:
# df = filechecker_to_df(file)
main_df = filechecker_to_df(file)

In [ ]:
df = main_df.loc[main_df['directory'].str.contains("Datenban")]

In [ ]:
# all_models = fetch_models('archiv')
all_models = [x for x in fetch_models('archiv')if 'Datenbase' in x.__name__] # add missing dbs 2020-12-03

In [ ]:
all_models

In [ ]:
for i, row in df.iterrows():
    leg_id = row['filename'].lower()
    leg_id = leg_id.split('.')[0]
    my_obj = find_matching_objects(all_models, leg_id)
    if my_obj is None:
        continue
    else:
        for prop in row.index:
            my_prop = f"fc_{prop}"
            my_val = f"{row[prop]}"
            setattr(my_obj, my_prop, my_val)
        my_obj.fc_match = True
        my_obj.save()

In [ ]:
items = Inventorybooks.objects.all()

In [ ]:
items.count()

In [ ]:
items.filter(fc_match=True).count()

In [ ]:
def fc_match_quota(app_name):
    summary = [
        ["class_name", 'all_objects', 'matching_resource', 'no_matching_resource', 'percentage']
    ]
    all_models = fetch_models(app_name)
    for x in all_models:
        items = x.objects.all()
        all_objects = items.count()
        matches = x.objects.filter(fc_match=True).count()
        misses = x.objects.exclude(fc_match=True).count()
        class_name = x.__name__
        try:
            percentage = (matches / all_objects)*100 
        except ZeroDivisionError:
            percentage = 0
        summary.append(
            [class_name, all_objects, matches, misses, percentage]
        )
    return summary

In [ ]:
table = fc_match_quota('archiv')

In [ ]:
df = pd.DataFrame(table, columns=table[0])

In [ ]:
df

In [ ]:
df.to_csv('matching_binaries.csv')